<a href="https://colab.research.google.com/github/apester/IME/blob/main/Lab_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# 1. Generate Synthetic Dataset (for demonstration purposes)
def generate_dataset(num_samples=1000, img_size=64):
    X = np.random.rand(num_samples, img_size, img_size, 1)
    y = np.random.randint(0, 2, num_samples)  # 0: good, 1: defect
    # Simulate defects by inserting brighter spots
    for i in range(num_samples):
        if y[i] == 1:  # defect
            x_coord, y_coord = np.random.randint(0, img_size, 2)
            X[i, x_coord:x_coord+5, y_coord:y_coord+5] += 0.8
            X[i] = np.clip(X[i], 0, 1)
    return X, y

X, y = generate_dataset()

# Split the dataset into train/test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# 2. Build a Simple CNN Model
model = models.Sequential([
    layers.Conv2D(16, (3, 3), activation='relu', input_shape=(64, 64, 1)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # binary classification
])

In [ ]:
# 3. Compile the Model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# 4. Train the Model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# 5. Evaluate Model Performance
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'\nTest accuracy: {test_acc:.2f}')

In [ ]:
# 6. Predict and Visualize Results
predictions = (model.predict(X_test[:5]) > 0.5).astype("int32")

fig, axes = plt.subplots(1, 5, figsize=(15,3))
for i, ax in enumerate(axes):
    ax.imshow(X_test[i].reshape(64,64), cmap='gray')
    ax.set_title(f'True: {y_test[i]} | Pred: {predictions[i][0]}')
    ax.axis('off')
plt.show()
